In [1]:
import itertools
import gensim
from gifter.modeling.data import lemmatized_frame
from gifter.modeling.tokenizer import lemmatize
from gensim import corpora
from nltk.corpus import brown

In [2]:
def LDA(texts=["default"], num=2, passes=100):
    if (texts == ["default"]):
        df = lemmatized_frame("../data.json", with_tags=False)
        texts = [df['lemmas'].irow(i) for i in range(df.shape[0])]
    else:
        texts2 = []
        for d in texts:
            texts2 = texts2 + [lemmatize(d, with_tags=False)]
            texts = texts2
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    #print "corpus: " + str(corpus)
    corpora.MmCorpus.serialize('./corpus.mm', corpus)
    mm_corpus = corpora.MmCorpus('./corpus.mm')
    #print "mm_corpus: " + str(mm_corpus)
    id2word = {}
    for word in dictionary.token2id:
        id2word[dictionary.token2id[word]] = word
    #lda = gensim.models.ldamodel.LdaModel(
    lda = gensim.models.ldamulticore.LdaMulticore(
        corpus=mm_corpus,
        num_topics=num,
        id2word=id2word,
        #update_every=1,
        eval_every=1,
        passes=passes,
        workers=1
    )
    for i in range(0, lda.num_topics):
        print "Topic number " + str(i) + " consists of words : " + lda.print_topic(i)
    return lda, dictionary

In [3]:
l, dic = LDA()

Topic number 0 consist of words : 0.045*president + 0.042*obama + 0.028*climate + 0.019*act + 0.015*change + 0.010*day + 0.009*american + 0.009*time + 0.008*http + 0.008*obamacare
Topic number 1 consist of words : 0.064*president + 0.053*obama + 0.015*selma + 0.015*work + 0.014*college + 0.013*watch + 0.010*march + 0.010*job + 0.010*year + 0.009*talk


In [4]:
documents = [
    "Apple is releasing a new product",
    "I love apple",
    "Amazon sells many things",
    "Microsoft announces Nokia acquisition",
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey"
]

D = [
    "I like to eat broccoli and bananas.",
    "I ate a banana and spinach smoothie for breakfast.",
    "Chinchillas and kittens are cute.",
    "My sister adopted a kitten yesterday.",
    "Look at this cute hamster munching on a piece of broccoli."
 ]

In [5]:
l, dic = LDA(texts=D, num=2)

Topic number 0 consist of words : 0.166*kitten + 0.100*cute + 0.099*yesterday + 0.099*sister + 0.099*adopt + 0.099*chinchilla + 0.034*broccoli + 0.034*eat + 0.034*banana + 0.034*piece
Topic number 1 consist of words : 0.114*eat + 0.114*banana + 0.114*broccoli + 0.068*spinach + 0.068*smoothie + 0.068*breakfast + 0.068*piece + 0.068*munch + 0.068*look + 0.068*hamster


In [6]:
l, dic = LDA(texts=documents, num=2)

Topic number 0 consist of words : 0.075*user + 0.056*time + 0.056*response + 0.056*apple + 0.035*survey + 0.033*relation + 0.033*error + 0.033*measurement + 0.033*perceive + 0.033*opinion
Topic number 1 consist of words : 0.060*tree + 0.060*graph + 0.043*human + 0.043*minor + 0.042*interface + 0.026*width + 0.026*order + 0.026*iv + 0.026*quasi + 0.026*machine


In [7]:
l, dic = LDA(texts=brown.words(), num=40, passes=1)

Topic number 0 consist of words : 0.130*tell + 0.116*thing + 0.084*turn + 0.069*woman + 0.029*job + 0.029*field + 0.025*line + 0.024*bit + 0.022*air + 0.021*clear
Topic number 1 consist of words : 0.099*mr + 0.074*night + 0.062*live + 0.032*rest + 0.025*early + 0.017*coffee + 0.017*force + 0.016*sick + 0.014*suit + 0.013*studio
Topic number 2 consist of words : 0.032*change + 0.022*hate + 0.016*lose + 0.014*eat + 0.013*freddy + 0.012*layer + 0.012*blanche + 0.011*subject + 0.010*pet + 0.010*doll
Topic number 3 consist of words : 0.209*man + 0.055*miss + 0.043*smile + 0.035*book + 0.033*minute + 0.026*cousin + 0.023*finger + 0.022*pretty + 0.020*ready + 0.016*paint
Topic number 4 consist of words : 0.218*look + 0.126*way + 0.078*sit + 0.035*half + 0.032*catch + 0.031*meet + 0.020*heavy + 0.019*church + 0.017*write + 0.016*clearly
Topic number 5 consist of words : 0.223*know + 0.067*place + 0.062*open + 0.054*stand + 0.042*set + 0.041*grow + 0.032*speak + 0.026*realize + 0.024*touch + 0.

In [17]:
def find_topic(new_doc="User", dictionary=dic, lda=l):
    if (new_doc == "User"):
        df = lemmatized_frame("../data.json", with_tags=False)
        new_doc = list(
            itertools.chain(
                *[df['lemmas'].irow(i) for i in range(df.shape[0])]
            )
        )
    else:
        new_doc = new_doc.lower().split()
    new_vec = dictionary.doc2bow(new_doc)
    vec_lda = sorted(lda[new_vec], key=lambda vec: vec[1], reverse=True)
    print str(vec_lda)
    print "Top topic is topic number " + str(
        vec_lda[0][0]
    ) + " consists of words : " + lda.print_topic(vec_lda[0][0])
    return vec_lda


In [18]:
%time find_topic()

[(37, 0.10770201168570052), (35, 0.072113714853879601), (18, 0.045270958023020565), (26, 0.043610604948622574), (0, 0.041844140845861279), (13, 0.037362392113134038), (4, 0.033857392688869038), (33, 0.030330232861490027), (2, 0.030270737541640273), (30, 0.030212396755356221), (7, 0.027490654492517745), (8, 0.02540333367637548), (14, 0.024759657787775797), (15, 0.024305337142576752), (24, 0.023825393624853961), (12, 0.023805179718994183), (27, 0.023774074024676489), (1, 0.022649045654128168), (34, 0.022443666504283678), (20, 0.021084373805999604), (5, 0.02064703218059941), (19, 0.019670802197358922), (17, 0.019261455409148837), (21, 0.016494188031877143), (6, 0.015780067487399685), (9, 0.015659095963112598), (11, 0.015601231684766835), (23, 0.015127426585328235), (32, 0.015002909256486622), (39, 0.014669656779502348), (22, 0.01463724232061368), (3, 0.013753453902696472), (38, 0.013734268880740688), (29, 0.013569990849101817), (16, 0.013273449229459064), (31, 0.012801310448500712), (10, 

[(37, 0.10770201168570052),
 (35, 0.072113714853879601),
 (18, 0.045270958023020565),
 (26, 0.043610604948622574),
 (0, 0.041844140845861279),
 (13, 0.037362392113134038),
 (4, 0.033857392688869038),
 (33, 0.030330232861490027),
 (2, 0.030270737541640273),
 (30, 0.030212396755356221),
 (7, 0.027490654492517745),
 (8, 0.02540333367637548),
 (14, 0.024759657787775797),
 (15, 0.024305337142576752),
 (24, 0.023825393624853961),
 (12, 0.023805179718994183),
 (27, 0.023774074024676489),
 (1, 0.022649045654128168),
 (34, 0.022443666504283678),
 (20, 0.021084373805999604),
 (5, 0.02064703218059941),
 (19, 0.019670802197358922),
 (17, 0.019261455409148837),
 (21, 0.016494188031877143),
 (6, 0.015780067487399685),
 (9, 0.015659095963112598),
 (11, 0.015601231684766835),
 (23, 0.015127426585328235),
 (32, 0.015002909256486622),
 (39, 0.014669656779502348),
 (22, 0.01463724232061368),
 (3, 0.013753453902696472),
 (38, 0.013734268880740688),
 (29, 0.013569990849101817),
 (16, 0.013273449229459064),

In [19]:
find_topic(new_doc = "I love watching films with my family and friends. I will write review of the film we watched yesterday")

[(1, 0.14991535815791426), (29, 0.14978780243077047), (19, 0.14961452069074083), (4, 0.14953320319171437), (18, 0.1493942791292022), (0, 0.14550483639965719)]
Top topic is topic number 1 consist of words : 0.099*mr + 0.074*night + 0.062*live + 0.032*rest + 0.025*early + 0.017*coffee + 0.017*force + 0.016*sick + 0.014*suit + 0.013*studio


[(1, 0.14991535815791426),
 (29, 0.14978780243077047),
 (19, 0.14961452069074083),
 (4, 0.14953320319171437),
 (18, 0.1493942791292022),
 (0, 0.14550483639965719)]